In [2]:
import tensorflow as tf
import numpy as np
import os
import time

In [3]:
dataset= open('./data/Harry_Potter_all_books_preprocessed.txt', 'r',encoding='utf-8').read()
print ('Length of text: {} characters'.format(len(dataset)))

Length of text: 5991293 characters


In [4]:
print(dataset[:300])

THE BOY WHO LIVED Mr and Mrs Dursley of number four Privet Drive were proud to say that they were perfectly normal thank you very much .They were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense .Mr Dursley was the director 


In [5]:
vocab = sorted(set(dataset))
print ('{} unique characters'.format(len(vocab)))

71 unique characters


In [6]:
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)

text_as_int = np.array([char2index[c] for c in dataset])

print(text_as_int)

[33 21 18 ...  0  2  0]


In [7]:
print ('{} -- characters mapped to int -- > {}'.format(repr(dataset[:13]), text_as_int[:13]))

'THE BOY WHO L' -- characters mapped to int -- > [33 21 18  0 15 28 38  0 36 21 28  0 25]


In [8]:
seq_length = 100
examples_per_epoch = len(dataset)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(index2char[i.numpy()])

T
H
E
 
B


2024-06-07 12:30:23.499571: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [5991293]
	 [[{{node Placeholder/_0}}]]


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(index2char[item.numpy()])))

'THE BOY WHO LIVED Mr and Mrs Dursley of number four Privet Drive were proud to say that they were per'
'fectly normal thank you very much .They were the last people youd expect to be involved in anything s'
'trange or mysterious because they just didnt hold with such nonsense .Mr Dursley was the director of '
'a firm called Grunnings which made drills .He was a big beefy man with hardly any neck although he di'
'd have a very large mustache .Mrs Dursley was thin and blonde and had nearly twice the usual amount o'


2024-06-07 12:30:23.559322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [5991293]
	 [[{{node Placeholder/_0}}]]


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 300 #256

# Number of RNN units
rnn_units1 = 512 #1024
rnn_units2 = 256
rnn_units=[rnn_units1, rnn_units2]
print(vocab_size)

71


In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [14]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

2024-06-07 12:30:23.779527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-07 12:30:23.780279: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-07 12:30:23.781266: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 300)           21300     
                                                                 
 gru (GRU)                   (64, None, 512)           1250304   
                                                                 
 gru_1 (GRU)                 (64, None, 256)           591360    
                                                                 
 dense (Dense)               (64, None, 71)            18247     
                                                                 
Total params: 1,881,211
Trainable params: 1,881,211
Non-trainable params: 0
_________________________________________________________________


In [16]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [17]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [18]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [19]:
EPOCHS=50

In [20]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50


2024-06-07 12:30:24.154132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [5991293]
	 [[{{node Placeholder/_0}}]]
2024-06-07 12:30:24.154294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [5991293]
	 [[{{node Placeholder/_0}}]]
2024-06-07 12:30:24.243211: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_d

926/926 [==============================] - 488s 525ms/step - loss: 1.6681 - accuracy: 0.5156
Epoch 2/50
926/926 [==============================] - 508s 547ms/step - loss: 1.2477 - accuracy: 0.6262
Epoch 3/50
926/926 [==============================] - 518s 558ms/step - loss: 1.1836 - accuracy: 0.6437
Epoch 4/50
926/926 [==============================] - 506s 545ms/step - loss: 1.1512 - accuracy: 0.6529
Epoch 5/50
926/926 [==============================] - 505s 545ms/step - loss: 1.1302 - accuracy: 0.6590
Epoch 6/50
926/926 [==============================] - 529s 570ms/step - loss: 1.1152 - accuracy: 0.6630
Epoch 7/50
926/926 [==============================] - 562s 606ms/step - loss: 1.1034 - accuracy: 0.6665
Epoch 8/50
926/926 [==============================] - 793s 856ms/step - loss: 1.0937 - accuracy: 0.6694
Epoch 9/50
926/926 [==============================] - 569s 613ms/step - loss: 1.0859 - accuracy: 0.6717
Epoch 10/50
926/926 [==============================] - 693s 748ms/step - lo

In [21]:
latest_check= tf.train.latest_checkpoint(checkpoint_dir)

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(latest_check)

model.build(tf.TensorShape([1, None]))

2024-06-07 21:01:28.239288: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-07 21:01:28.240169: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-07 21:01:28.240656: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 300)            21300     
                                                                 
 gru_2 (GRU)                 (1, None, 512)            1250304   
                                                                 
 gru_3 (GRU)                 (1, None, 256)            591360    
                                                                 
 dense_1 (Dense)             (1, None, 71)             18247     
                                                                 
Total params: 1,881,211
Trainable params: 1,881,211
Non-trainable params: 0
_________________________________________________________________


In [24]:
def generate_text(model, start_string):

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low results in more predictable text.
  # Higher results in more surprising text.
  # Experiment to find the best setting.
  scaling = 0.5 #1

  # batch size == 1
  
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / scaling
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(index2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [25]:
print(generate_text(model, start_string=u"Hermoine was unable to understand"))

Hermoine was unable to understand that he was supposed to be difficult to decide the scene was still tiny the rest of the window .The end of the room which was now burned in the firelight .I dont think so said Dumbledore .I dont think help you to come back to the potion when he was talking about said Harry .Were going to get to the school and that Harry was sure that they wouldve left the dungeon than the others were still the back to the back and the silver handkerchief and screaming with a roar of furry gate in his fingers and fell asleep with a twisted skin made him as he had drawn rows of firewhisky .Harry was staring at him .Its a lot of people are again in the school when they had left the pictures of Inferi to get the hall .Why did they ?Harry saw the last of the others the only thing at all .I believe that she knew what was happening to make him feel over the last time .He had been hoping to be stringed around the room he could not answer .I wonder what we had to go back to the